In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import os

In [9]:
data = pd.read_csv('C:/Users/Stark/OneDrive/Desktop/Final Year Project/indiana_reports.csv')

In [11]:
projection_data = pd.read_csv('C:/Users/Stark/OneDrive/Desktop/Final Year Project/indiana_projections.csv')

In [13]:
#Drop unnecessary columns – image and comparison
#projection already tells us what type of image – so image unncessary
data_dropped_cols = data.drop(columns=['image','Problems','comparison'])

#Drop rows with NaNs on both findings and impressions
data_dropped_nans = data_dropped_cols.dropna(subset=['findings','impression'], how='all')

In [15]:
data_dropped_nans['back_info'] = data_dropped_nans[['MeSH','indication']].astype('str').agg(' '.join, axis=1)

C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\3220265758.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dropped_nans['back_info'] = data_dropped_nans[['MeSH','indication']].astype('str').agg(' '.join, axis=1)


In [17]:
#Append the impression to findings to make new caption column
data_dropped_nans['caption'] = data_dropped_nans[['findings','impression']].astype('str').agg(' '.join, axis=1)

print(f"Dropped {len(data)-len(data_dropped_nans)} rows from {len(data)} rows. New row count: {len(data_dropped_nans)}")

Dropped 25 rows from 3851 rows. New row count: 3826


C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\1853989472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dropped_nans['caption'] = data_dropped_nans[['findings','impression']].astype('str').agg(' '.join, axis=1)


In [19]:
#Replacing the nan values
data_dropped_nans['findings'] = data_dropped_nans['findings'].fillna('no findings')
data_dropped_nans['impression'] = data_dropped_nans['impression'].fillna('no impression')

C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\2211554902.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dropped_nans['findings'] = data_dropped_nans['findings'].fillna('no findings')
C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\2211554902.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dropped_nans['impression'] = data_dropped_nans['impression'].fillna('no impression')


In [21]:
#Clean MeSH and Problems
data_dropped_nans['MeSH'] = data_dropped_nans['MeSH'].str.replace('/',' ').str.replace(';',',')

# CLEAN ALL TEXT COLUMNS EXCEPT FILENAME
cols_to_clean = ['MeSH','indication','findings','impression','caption']

#Remove leading and trailing commas and fullstops, etc.
data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace('(^[.,;/:\s]+)|([.,;/:\s]+$)', '', regex=True)

#Clear excess whitespace between words and punctuation
data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace(r'\s+([,?.!;"])',r'\1', regex=True)

#Clean out left over gender with .....
data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace('(male|female)? with','',regex=True)

#Get rid of numbering in impression or findings
data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace('\d+[.]',"", regex=True)

data_dropped_nans.caption[1]

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\658785991.py:8: SyntaxWarning: invalid escape sequence '\s'
  data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace('(^[.,;/:\s]+)|([.,;/:\s]+$)', '', regex=True)
C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\658785991.py:17: SyntaxWarning: invalid escape sequence '\d'
  data_dropped_nans[cols_to_clean] = data_dropped_nans[cols_to_clean].replace('\d+[.]',"", regex=True)
C:\Users\Stark\AppData\Local\Temp\ipykernel_5752\658785991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

'Borderline cardiomegaly. Midline sternotomy XXXX. Enlarged pulmonary arteries. Clear lungs. Inferior XXXX XXXX XXXX. No acute pulmonary findings'

In [22]:
data_dropped_nans

,uid,MeSH,indication,findings,impression,back_info,caption
0,1,normal,Positive TB test,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX,normal Positive TB test,The cardiac silhouette and mediastinum size ar...
1,2,"Cardiomegaly borderline,Pulmonary Artery enlarged",Preop bariatric surgery,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings,Cardiomegaly/borderline;Pulmonary Artery/enlar...,Borderline cardiomegaly. Midline sternotomy XX...
2,3,normal,"rib pain after a XXXX, XXXX XXXX steps this XX...",no findings,"No displaced rib fractures, pneumothorax, or p...","normal rib pain after a XXXX, XXXX XXXX steps ...","nan No displaced rib fractures, pneumothorax, ..."
3,4,"Pulmonary Disease, Chronic Obstructive,Bullous...",XXXX-year-old XXXX XXXX,There are diffuse bilateral interstitial and a...,Bullous emphysema and interstitial fibrosis. ...,"Pulmonary Disease, Chronic Obstructive;Bullous...",There are diffuse bilateral interstitial and a...
4,5,"Osteophyte thoracic vertebrae multiple small,T...",Chest and nasal congestion,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality,Osteophyte/thoracic vertebrae/multiple/small;T...,The cardiomediastinal silhouette and pulmonary...
...,...,...,...,...,...,...,...
3846,3995,"Lung hyperdistention mild,Diaphragm bilateral ...","Nausea, vomiting x2 weeks. Dialysis patient",The cardiomediastinal silhouette and pulmonary...,Interval resolution of bibasilar airspace dis...,Lung/hyperdistention/mild;Diaphragm/bilateral/...,The cardiomediastinal silhouette and pulmonary...
3847,3996,Spine degenerative,NaN,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormality,Spine/degenerative nan,The lungs are clear. Heart size is normal. No ...
3848,3997,"Opacity lung upper lobe right round small,Gran...",XXXX-year-old positive PPD,"Heart sizein normal limits. Small, nodular opa...","No acute findings, no evidence for active TB",Opacity/lung/upper lobe/right/round/small;Gran...,"Heart sizein normal limits. Small, nodular opa..."
3849,3998,normal,tuberculosis positive PPD,no findings,Heart size is normal and the lungs are clear,normal tuberculosis positive PPD,nan Heart size is normal and the lungs are clear


In [25]:
cleaned_data = data_dropped_nans.drop(columns=['MeSH','indication','findings','impression'])

In [27]:
cleaned_data

,uid,back_info,caption
0,1,normal Positive TB test,The cardiac silhouette and mediastinum size ar...
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlar...,Borderline cardiomegaly. Midline sternotomy XX...
2,3,"normal rib pain after a XXXX, XXXX XXXX steps ...","nan No displaced rib fractures, pneumothorax, ..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...",There are diffuse bilateral interstitial and a...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,The cardiomediastinal silhouette and pulmonary...
...,...,...,...
3846,3995,Lung/hyperdistention/mild;Diaphragm/bilateral/...,The cardiomediastinal silhouette and pulmonary...
3847,3996,Spine/degenerative nan,The lungs are clear. Heart size is normal. No ...
3848,3997,Opacity/lung/upper lobe/right/round/small;Gran...,"Heart sizein normal limits. Small, nodular opa..."
3849,3998,normal tuberculosis positive PPD,nan Heart size is normal and the lungs are clear


In [29]:
merged_data = pd.merge(cleaned_data, projection_data, on='uid', how='inner')

# Display the merged dataset
print(merged_data.head())

   uid                                          back_info  \
0    1                            normal Positive TB test   
1    1                            normal Positive TB test   
2    2  Cardiomegaly/borderline;Pulmonary Artery/enlar...   
3    2  Cardiomegaly/borderline;Pulmonary Artery/enlar...   
4    3  normal rib pain after a XXXX, XXXX XXXX steps ...   

                                             caption                filename  \
0  The cardiac silhouette and mediastinum size ar...  1_IM-0001-4001.dcm.png   
1  The cardiac silhouette and mediastinum size ar...  1_IM-0001-3001.dcm.png   
2  Borderline cardiomegaly. Midline sternotomy XX...  2_IM-0652-1001.dcm.png   
3  Borderline cardiomegaly. Midline sternotomy XX...  2_IM-0652-2001.dcm.png   
4  nan No displaced rib fractures, pneumothorax, ...  3_IM-1384-1001.dcm.png   

  projection  
0    Frontal  
1    Lateral  
2    Frontal  
3    Lateral  
4    Frontal  


In [31]:
# Save the cleaned dataset to a CSV file
merged_data.to_csv('merged_dataset.csv', index=False)